# Ordinary Least Squares Regression

This notebook demonstrates how to perform Ordinary Least Squares (OLS) regression using the `statsmodels` library in Python. We will fit a linear regression model to the cherry blossom dataset and interpret the results.


## Plan and assumptions

- Use all available predictors except `year` and `bloom_day` as features.
- Use years 1921–2015 as the pool for training/validation. The user requested an "80/10 split"; that seems underspecified, so I assume a sequential 80/20 split (train/validation) within 1921–2015 to keep time order.
- Reserve 2016–2025 as the test set and report performance on that set.
- Metrics: RMSE, MAE, and WMAPE (defined here as sum(|error|) / sum(|actual|)).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from sklearn.metrics import mean_squared_error, mean_absolute_error

sns.set(style='whitegrid')


## 1) Load data

In [3]:
DATA_PATH = '../data/cherry_blossom_data.csv'
df = pd.read_csv(DATA_PATH)
print('Loaded rows:', len(df))
df.head()


Loaded rows: 107


,year,days_dec_ge_45,days_jan_ge_45,days_feb_ge_45,prec_winter,mean_temp_winter,surface_temp_chg,climate_incidents,bloom_day
0,1921,30,24,23,3.41,42.8,-0.05,3,79
1,1922,20,17,19,2.56,39.1,-0.12,2,97
2,1923,18,15,14,3.12,36.4,-0.18,1,99
3,1924,25,22,25,3.68,43.2,-0.09,4,104
4,1925,16,12,14,2.89,36.8,-0.14,2,86


## 2) Prepare features and target

In [4]:
# Ensure numeric types
numeric_cols = ['days_dec_ge_45','days_jan_ge_45','days_feb_ge_45','prec_winter',
                'mean_temp_winter','surface_temp_chg','climate_incidents']
for c in numeric_cols + ['bloom_day','year']:
    df[c] = pd.to_numeric(df[c], errors='coerce')

# Drop rows with missing essential values
df = df.dropna(subset=['year','bloom_day'] + numeric_cols).reset_index(drop=True)

# Features and target
FEATURES = numeric_cols
TARGET = 'bloom_day'


## 3) Create train / validation / test splits

In [5]:
# Training/validation pool: 1921-2015 (inclusive)
train_val_df = df[(df['year'] >= 1921) & (df['year'] <= 2015)].sort_values('year').reset_index(drop=True)
# Test set: 2016-2025
test_df = df[(df['year'] >= 2016) & (df['year'] <= 2025)].sort_values('year').reset_index(drop=True)

print(f'Train/Val rows: {len(train_val_df)}, Test rows: {len(test_df)}')

# Sequential split (time-ordered) within train_val: 80% train, 20% val
n = len(train_val_df)
if n == 0:
    raise ValueError('No data in 1921-2015 range to train/validate on.')
train_n = int(np.floor(n * 0.8))
train_df = train_val_df.iloc[:train_n].reset_index(drop=True)
val_df = train_val_df.iloc[train_n:].reset_index(drop=True)

print(f'Train rows: {len(train_df)}, Val rows: {len(val_df)}')


Train/Val rows: 97, Test rows: 10
Train rows: 77, Val rows: 20


## 4) Fit OLS on training data

In [6]:
X_train = train_df[FEATURES]
y_train = train_df[TARGET]
X_train_const = sm.add_constant(X_train)

ols_model = sm.OLS(y_train, X_train_const).fit()
print(ols_model.summary())


                            OLS Regression Results                            
Dep. Variable:              bloom_day   R-squared:                       0.129
Model:                            OLS   Adj. R-squared:                  0.041
Method:                 Least Squares   F-statistic:                     1.460
Date:                Tue, 18 Nov 2025   Prob (F-statistic):              0.196
Time:                        23:32:13   Log-Likelihood:                -255.14
No. Observations:                  77   AIC:                             526.3
Df Residuals:                      69   BIC:                             545.0
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                99.7447     26.89

## 5) Validation diagnostics (optional)

In [7]:
X_val = val_df[FEATURES]
y_val = val_df[TARGET]
val_pred = ols_model.predict(sm.add_constant(X_val))

val_rmse = np.sqrt(mean_squared_error(y_val, val_pred))
val_mae = mean_absolute_error(y_val, val_pred)
val_wmape = np.sum(np.abs(y_val - val_pred)) / np.sum(np.abs(y_val))

print('Validation RMSE: {:.3f}'.format(val_rmse))
print('Validation MAE: {:.3f}'.format(val_mae))
print('Validation WMAPE: {:.3%}'.format(val_wmape))


Validation RMSE: 6.635
Validation MAE: 5.136
Validation WMAPE: 5.638%


## 6) Evaluate on test set (2016-2025)

In [8]:
if len(test_df) == 0:
    print('No test data available for 2016-2025 range.')
else:
    X_test = test_df[FEATURES]
    y_test = test_df[TARGET]
    X_test_const = sm.add_constant(X_test)
    y_pred_test = ols_model.predict(X_test_const)

    test_rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
    test_mae = mean_absolute_error(y_test, y_pred_test)
    test_wmape = np.sum(np.abs(y_test - y_pred_test)) / np.sum(np.abs(y_test))

    print('\nTest set performance (2016-2025)')
    print('Rows:', len(test_df))
    print('RMSE: {:.3f}'.format(test_rmse))
    print('MAE: {:.3f}'.format(test_mae))
    print('WMAPE: {:.3%}'.format(test_wmape))



Test set performance (2016-2025)
Rows: 10
RMSE: 5.939
MAE: 4.601
WMAPE: 5.426%


## 7) Plot actuals (blue) vs forecasts (red) on the test set

In [ ]:
# Create a combined plot: actuals from the full dataset and forecasts overlay for 2016+
full_years = df['year']
full_actuals = df[TARGET]

plt.figure(figsize=(15, 5))
plt.plot(full_years, full_actuals.values, marker='o', color='tab:blue', label='Actual (since 1921)')

if len(test_df) > 0:
    plt.plot(test_df['year'], y_pred_test, marker='o', color='tab:red', label='Forecast (2016-2025)')
    plt.axvline(2016, color='k', linestyle='--', linewidth=1, label='Forecast start (2016)')

plt.xlabel('Year')
plt.ylabel('Bloom Day')
plt.title('Actual vs Forecast - Full Series with Forecasts (2016-2025)')
plt.legend()
plt.grid(True)
plt.show()


## 8) Notes and next steps

- We fit a simple linear OLS model using all provided numeric predictors. The model summary above shows coefficients, p-values, and overall fit statistics (R-squared, etc.).
- If you want improved predictive performance, consider: feature engineering, regularization (Ridge/Lasso), tree-based models, or time-series models that explicitly model temporal dependence.
- I assumed an 80/20 sequential split for train/validation inside 1921–2015 to preserve time ordering. If you prefer a different split (e.g., 80/10/10 or cross-validation), tell me and I will update the notebook.
